In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data

In [ ]:
def get_tblogs(data_id):
    root_dir = "/home/ubuntu/ray_results/PPO"
    exp_path = glob.glob(os.path.join(root_dir, "*" + data_id + "*", "event*"))
    if len(exp_path) > 1:
        print("Error: more than one experiments with that id found")
    df = tflog2pandas(exp_path[0])
    df = df.pivot(index="step", columns="metric")
    df_mean = df.filter(regex=(".*(custom_metrics|reward).*_mean"))
    df_mean = df_mean.rename(columns={x: x[24:-5] if "custom_metrics" in x else x[9:-5] for _,x in df.columns})
    return df_mean

In [ ]:
def gen_plot_func(regex, title="", xlabel="", ylabel="", ylim=None, figure=True):
    def plot_func(df):
        if figure:
            plt.figure()
        plt.plot(df.filter(regex=(regex)))
        plt.ylim(ylim)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
    return plot_func

In [ ]:
def step_average(a, steps):
    b = np.zeros(a.shape)
    c = 0
    for i in range(len(a)):
        c += a[i]
        if i >= steps:
            c -= a[i-steps]
        b[i] = c / min(steps, i+1)
    return b

In [ ]:
df_rl = get_tblogs("9db51")

In [ ]:
df_alg = get_tblogs("23591")

In [ ]:
df_rl = df_rl[df_rl.index <= 2000000]
df_alg = df_alg[df_alg.index <= 2000000]

dfs = [df_rl, df_alg]

In [ ]:
alg_crah_fan.mean()

In [ ]:
# RL data
rl_compressor = df_rl.filter(regex=".*power/compressor").to_numpy()
rl_server_fan = df_rl.filter(regex=".*power/server_fan").to_numpy()
rl_crah_fan = df_rl.filter(regex=".*power/crah_fan").to_numpy()
rl_server_load = df_rl.filter(regex=".*power/total_server_load").to_numpy()
rl_energy = df_rl.filter(regex=".*cost/energy").to_numpy() * 100
rl_dropped = df_rl.filter(regex=".*job/dropped").to_numpy() 

rl_time = df_rl.index.to_numpy()

rl_total_cooling_power_kw = (rl_compressor+rl_server_fan+rl_crah_fan) / 1000
rl_total_power_kw = (rl_compressor+rl_server_fan+rl_crah_fan+rl_server_load) / 1000
rl_it_power_kw = (rl_server_fan+rl_server_load) / 1000
rl_pue = rl_total_power_kw / rl_it_power_kw

# Alg data
alg_compressor = df_alg.filter(regex=".*power/compressor").to_numpy()
alg_server_fan = df_alg.filter(regex=".*power/server_fan").to_numpy()
alg_crah_fan = df_alg.filter(regex=".*power/crah_fan").to_numpy()
alg_server_load = df_alg.filter(regex=".*power/total_server_load").to_numpy()
alg_energy = df_alg.filter(regex=".*cost/energy").to_numpy() * 100
alg_dropped = df_alg.filter(regex=".*job/dropped").to_numpy() 

alg_time = df_alg.index.to_numpy()

alg_total_cooling_power_kw = (alg_compressor+alg_server_fan+alg_crah_fan) / 1000
alg_total_power_kw = (alg_compressor+alg_server_fan+alg_crah_fan+alg_server_load) / 1000
alg_it_power_kw = (alg_server_fan+alg_server_load) / 1000
alg_pue = alg_total_power_kw / alg_it_power_kw

# Cooling power plot
plt.figure(figsize=(6,2))
plt.plot(rl_time, rl_total_cooling_power_kw)
plt.plot(alg_time, alg_total_cooling_power_kw)
plt.title("Total cooling power")
plt.xlabel("Iterations")
plt.ylabel("Power [kW]")
plt.legend(["RL", "Constant"])

# Total power plot
fix, ax = plt.subplots(2, figsize=(6,5))
ax[0].plot(rl_time, rl_total_power_kw)
ax[0].plot(alg_time, alg_total_power_kw)
ax[0].set_title("DC power")
ax[0].set(ylabel="Total power [kW]")
ax[0].label_outer()
ax[0].legend(["RL", "Constant"])

# PUE plot
ax[1].plot(rl_time, rl_pue)
ax[1].plot(alg_time, alg_pue)
ax[1].set(xlabel="Iterations", ylabel="PUE")
#ax[0].ylabel("PUE")
#ax[0].legend(["RL", "Constant"])

# Compressor plot
plt.figure(figsize=(6,2))
plt.plot(rl_time, rl_compressor / 1000)
plt.plot(alg_time, alg_compressor / 1000)
plt.title("Compressor power")
plt.xlabel("Iterations")
plt.ylabel("Power [kW]")
plt.legend(["RL", "Constant"])

# Dropped plot
plt.figure(figsize=(6,2))
plt.plot(rl_time, rl_dropped)
plt.plot(alg_time, alg_dropped)
plt.title("Average fraction of dropped jobs")
plt.xlabel("Iterations")
plt.ylabel("Fraction dropped")
plt.legend(["RL", "Constant"])


In [ ]:
list(map(gen_plot_func(".*cost/energy", figure=False, xlabel="Time [s]", ylabel="Power [W]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Cooling power")

In [ ]:
list(map(gen_plot_func(".*reward", ylim=[-30, 0], figure=False, xlabel="Time [s]", ylabel="Power [W]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Reward")

In [ ]:
list(map(gen_plot_func(".job/dropped", ylim=[0, 0.2], figure=False, xlabel="Time [s]", ylabel="Reward"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Fraction of jobs dropped")

In [ ]:
list(map(gen_plot_func(".*power/compressor", figure=False, xlabel="Time [s]", ylabel="Power [W]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Compressor")

In [ ]:
gen_plot_func(".*srv\d+/temp_cpu", xlabel="Time [s]", ylabel="Temperature [C]", title="CPU temperature")(dfs[0])

In [ ]:
df_rl.index

In [ ]:
rlload = df_rl.filter(regex=(".*srv\d+/load"))
rlmean = rlload.mean(axis=1).to_numpy()
rlstd = rlload.std(axis=1).to_numpy()
t = df_rl.index

plt.errorbar(t, rlmean, rlstd, linestyle='None', marker='^')

In [ ]:
list(map(gen_plot_func(".*srv\d+/load", xlabel="Time [s]", ylabel="Load [W]", title="Load distribution"), dfs));

In [ ]:
list(map(gen_plot_func(".crah0/temp_out", figure=False, xlabel="Time [s]", ylabel="Temperature [C]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("CRAH temperature setpoint")

In [ ]:
list(map(gen_plot_func(".crah0/flow", figure=False, xlabel="Time [s]", ylabel="Flow [$m^3/s$]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("CRAH flow setpoint")

In [ ]:
list(map(gen_plot_func(".srv/server_total_flow", figure=False, xlabel="Time [s]", ylabel="Flow [$m^3/s$]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Total flow through servers")